# Alpha 191 复现

In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time
import os

# 回测框架
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeed

# 基础函数
import utilsJ

## 策略

### 数据槽

In [2]:
class data_alpha191(btfeed.PandasData):
    '''
    The ``dataname`` parameter inherited from ``feed.DataBase`` is the pandas
    DataFrame
    '''
    lines = (
    )

    params = (
        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('datetime', None),

        # Possible values below:
        #  None : column not present
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', -1),
        ('volume', -1),
    )

### Alpha指标集

Alpha14 CLOSE-DELAY(CLOSE,5)

Alpha15 OPEN/DELAY(CLOSE,1)-1

Alpha18 CLOSE/DELAY(CLOSE,5)

Alpha20 (CLOSE-DELAY(CLOSE,6))/DELAY(CLOSE,6)*100

Alpha34 MEAN(CLOSE,12)/CLOSE

Alpha65 MEAN(CLOSE,6)/CLOSE

Alpha150 (CLOSE+HIGH+LOW)/3*VOLUME

Alpha168 (-1*VOLUME/MEAN(VOLUME,20))

In [3]:
class alpha14(bt.Indicator):
    lines = ('alpha14', )

    def __init__(self):
        self.lines.alpha14 = self.data.close - self.data.close(-5)
        super(alpha14, self).__init__()


class alpha15(bt.Indicator):
    lines = ('alpha15', )

    def __init__(self):
        self.lines.alpha15 = self.data.open / self.data.close(-1) - 1
        super(alpha15, self).__init__()


class alpha18(bt.Indicator):
    lines = ('alpha18', )

    def __init__(self):
        self.lines.alpha18 = self.data.close / self.data.close(-5)
        super(alpha18, self).__init__()


class alpha20(bt.Indicator):
    lines = ('alpha20', )

    def __init__(self):
        self.lines.alpha20 = (self.data.close - self.data.close(6)) / (self.data.close(-6) * 100)
        super(alpha20, self).__init__()


class alpha34(bt.Indicator):
    lines = ('alpha34', )

    def __init__(self):
        self.lines.alpha34 = btind.SimpleMovingAverage(self.data.close, period=12) / self.data.close
        super(alpha34, self).__init__()


class alpha65(bt.Indicator):
    lines = ('alpha65', )

    def __init__(self):
        self.lines.alpha65 = btind.SimpleMovingAverage(self.data.close, period=6) / self.data.close
        super(alpha65, self).__init__()


class alpha150(bt.Indicator):
    lines = ('alpha150', )

    def __init__(self):
        self.lines.alpha150 = (self.data.close + self.data.high + self.data.low) / (3 * self.data.volume)
        super(alpha150, self).__init__()


class alpha168(bt.Indicator):
    lines = ('alpha168', )

    def __init__(self):
        self.lines.alpha168 = -1 * self.data.volume / btind.SimpleMovingAverage(self.data.volume, period=20)
        super(alpha168, self).__init__()

### 策略主体

In [8]:
class alpha191(bt.Strategy):
    
    params = (
        ('printlog', False),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.p.printlog or doprint:
            dt = dt or self.data.datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        
        # Initialization
        self.buyprice = None
        self.sellprice = None
        
        self.alpha168 = alpha168()


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
                self.buyprice = order.executed.price

            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):
#        if self.order:
#            return
        #print(self.alpha168[0])
        return

## 单股回测

In [ ]:
startdate = dt.datetime(2020,1,1) - dt.timedelta(days=20)
enddate = dt.datetime(2020,12,31)
stock_code = '000166.SZ'


if __name__ ==  '__main__':

    # Initialization
    cerebro = bt.Cerebro()
    strats = cerebro.addstrategy(alpha191, printlog=True) 


    # Data
    df = utilsJ.get_stock(stock_code, startdate, enddate)
    data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)
    cerebro.adddata(data)

    # Start conditions
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(20000)
    #cerebro.broker.setcommission()
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Execution
    cerebro.run()

    # Final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))

    # Visualization
    cerebro.plot(iplot=False)